# Statistical models in R
This notebook is covering:
1. Data preprocessing:
    1. Aligning all dataframe to 5-day week from 2018-10-01 to 2024-08-30.
    2. Interpolating missing values.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

Removing duplicates and converting NBP to MWh / EUR

In [2]:
nbp = pd.read_csv("../data/raw/nbp.csv")
peg = pd.read_csv("../data/raw/peg.csv")
the = pd.read_csv("../data/raw/the.csv")
ttf = pd.read_csv("../data/raw/ttf.csv")
ztp = pd.read_csv("../data/raw/ztp.csv")
gbpeur = pd.read_csv("../data/raw/gbpeur.csv")
hubs = [nbp, peg, the, ttf, ztp, gbpeur]

In [3]:
for idx, hub in enumerate(hubs):
    hub["Date"] = pd.to_datetime(hub["Date"])
    hub.set_index("Date", inplace=True)
    hub = hub[~hub.index.duplicated(keep='first')]
    hubs[idx] = hub

nbp, peg, the, ttf, ztp, gbpeur = hubs

In [4]:
nbp_eur_mwh = nbp.copy()
nbp_eur_mwh['CLOSE'] = nbp_eur_mwh['CLOSE'] * gbpeur['CLOSE'] / 2.9307

In [5]:
nbp_eur_mwh[['CLOSE']].to_csv("../data/close/nbp_close.csv")
peg[['CLOSE']].to_csv("../data/close/peg_close.csv")
the[['CLOSE']].to_csv("../data/close/the_close.csv")
ttf[['CLOSE']].to_csv("../data/close/ttf_close.csv")
ztp[['CLOSE']].to_csv("../data/close/ztp_close.csv")

Continuing preprocessing in R

In [6]:
# rpy2 is a Python package that allows you to run R code from Python
%pip install rpy2

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Load the rpy2 extension to use R in Jupyter
%load_ext rpy2.ipython

The magic function `%%R` is used for running R code in Jupyter

Loading required libraries

In [8]:
%%R
# Install required packages
if (require("dplyr") == FALSE) {
  install.packages("dplyr")
  library(dplyr)
}
if (require("zoo") == FALSE) {
  install.packages("zoo")
  library(zoo)
}



Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



In [9]:
%%R
# Load data
hub_prices <- list(
  nbp = read.csv("../data/close/nbp_close.csv"),
  the = read.csv("../data/close/the_close.csv"),
  ttf = read.csv("../data/close/ttf_close.csv")
)

In [13]:
%%R
hubprices <- hub_prices$the
hubprices

           Date   CLOSE
1    2015-01-02  20.738
2    2015-01-05  20.442
3    2015-01-06  19.820
4    2015-01-07  19.913
5    2015-01-08  19.836
6    2015-01-09  19.750
7    2015-01-12  20.095
8    2015-01-13  20.686
9    2015-01-14  20.718
10   2015-01-15  20.397
11  

 2015-01-16  20.000
12   2015-01-19  19.489
13   2015-01-20  19.575
14   2015-01-21  19.782
15   2015-01-22  19.855
16   2015-01-23  19.820
17   2015-01-26  19.996
18   2015-01-27  20.276
19   2015-01-28  20.611
20   2015-01-29  20.986
21   2015-01-30  20.564
22   2015-02-02  20.700
23   2015-02-03  20.484
24   2015-02-04  20.732
25   2015-02-05  21.716
26   2015-02-06  21.464
27   2015-02-09  21.758
28   2015-02-10  22.275
29   2015-02-11  23.270
30   2015-02-12  24.395
31   2015-02-13  23.710
32   2015-02-16  23.061
33   2015-02-17  22.774
34   2015-02-18  22.867
35   2015-02-19  22.110
36   2015-02-20  22.509
37   2015-02-23  22.165
38   2015-02-24  22.617
39   2015-02-25  23.481
40   2015-02-26  23.888
41   2015-02-27  23.245
42   2015-03-02  22.551
43   2015-03-03  22.130
44   2015-03-04  22.335
45   2015-03-05  21.589
46   2015-03-06  21.335
47   2015-03-09  21.864
48   2015-03-10  21.890
49   2015-03-11  22.168
50   2015-03-12  21.885
51   2015-03-13  21.712
52   2015-03-16  21.

In [ ]:
%%R
start_date <- as.Date("2018-10-01")
end_date <- as.Date("2024-08-30")
date_seq <- seq.Date(start_date, end_date, by = "day")
date_index <- date_seq[!weekdays(date_seq) %in% c("Saturday", "Sunday")]

In [ ]:
%%R
missing_dates <- lapply(hub_prices, function(df) {
  df$Date <- as.Date(df$Date)
  df <- merge(data.frame(Date = date_index), df, by = "Date", all.x = TRUE)
  
  missing <- df %>%
    filter(is.na(CLOSE)) %>%
    select(Date)
  
  df <- df %>%
    mutate(CLOSE = na.approx(CLOSE, rule = 2))
  
  return(list(data = df, missing_dates = missing))
})

missing_dates_list <- lapply(missing_dates, function(x) x$missing_dates)
names(missing_dates_list) <- names(hub_prices) # Assign names for clarity
missing_dates_list

$nbp
         Date
1  2018-12-25
2  2019-01-01
3  2019-04-19
4  2019-04-22
5  2019-05-06
6  2019-05-27
7  2019-08-26
8  2019-12-25
9  2019-12-26
10 2020-01-01
11 2020-04-10
12 2020-12-25
13 2020-12-28
14 2021-01-01
15 2021-04-02
16 2021-04-05
17 2021-05-03
18 2021-05-31
19 2021-08-30
20 2021-12-27
21 2021-12-28
22 2022-01-03
23 2022-04-15
24 2022-04-18
25 2022-05-02
26 2022-06-02
27 2022-06-03
28 2022-08-29
29 2022-12-26
30 2022-12-27
31 2023-01-02
32 2023-04-07
33 2023-04-10
34 2023-05-01
35 2023-05-08
36 2023-05-29
37 2023-08-28
38 2023-12-25
39 2023-12-26
40 2024-01-01
41 2024-03-29
42 2024-04-01
43 2024-05-06
44 2024-05-27
45 2024-08-26

$the
         Date
1  2018-11-06
2  2018-11-07
3  2018-12-07
4  2018-12-25
5  2018-12-26
6  2019-01-01
7  2019-01-30
8  2019-04-19
9  2019-04-22
10 2019-05-01
11 2019-05-06
12 2019-05-27
13 2019-07-02
14 2019-07-03
15 2019-08-26
16 2019-09-04
17 2019-11-06
18 2019-11-07
19 2019-12-25
20 2019-12-26
21 2020-01-01
22 2020-04-10
23 2020-04-13
24 2020-0

In [ ]:
%%R
hub_prices <- lapply(hub_prices, function(df) {
  df$Date <- as.Date(df$Date)
  df <- merge(data.frame(Date = date_index), df, by = "Date", all.x = TRUE)
  df <- df %>%
    mutate(CLOSE = na.approx(CLOSE, rule = 2))
  return(df)
})

In [ ]:
%%R
nbp_price <- hub_prices$nbp
peg_price <- hub_prices$peg
the_price <- hub_prices$the
ttf_price <- hub_prices$ttf
ztp_price <- hub_prices$ztp

In [39]:
%%R
write.csv(nbp_price, "../data/interpolated/nbp_close_interpolated.csv", row.names = FALSE)
write.csv(peg_price, "../data/interpolated/peg_close_interpolated.csv", row.names = FALSE)
write.csv(the_price, "../data/interpolated/the_close_interpolated.csv", row.names = FALSE)
write.csv(ttf_price, "../data/interpolated/ttf_close_interpolated.csv", row.names = FALSE)
write.csv(ztp_price, "../data/interpolated/ztp_close_interpolated.csv", row.names = FALSE)